In [2]:
pip install selenium

     ---------------------------------------- 6.9/6.9 MB 5.3 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 4.8 MB/s eta 0:00:00
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import database

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException    
import re
import pandas as pd
import time



In [2]:
# specifying path und driver

PATH = r"C:\Users\grego\Dev\Data and IR\Abstimmungsverhalten_v2\chromedriver_win32.exe"
driver = webdriver.Chrome(PATH)

C:\Users\grego\AppData\Local\Temp\ipykernel_11292\610209268.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [3]:

# Set up the webdriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# Navigate to the relevant page
driver.get("https://www.bundestag.de/abgeordnete/biografien")

# Expand the page to show all MPs
button = driver.find_element(By.CSS_SELECTOR, '.bt-link-list[href="javascript:void(0);"]')
button.click()
time.sleep(5)

# Get the links of all MPs
try:
    links = [i.get_attribute('href') for i in driver.find_elements(By.CSS_SELECTOR, 'a[href^="/abgeordnete"]')]
    print('Success!!!')
    print(f'Found {len(links)} links.')
except Exception as e:
    print('Error:', e)

# Quit the webdriver
driver.quit()


Success!!!
Found 776 links.


In [4]:

start_time = time.time()

# working with the links liste generated above. we need to filter out the wrong links first before iterating over it and fetching the relevant information

regex = 'biografien\/[a-zA-Z]\/'

falsche_links = []
richtige_links = []

for lnk in links:
    if bool(re.search(regex, lnk)):
        richtige_links.append(lnk)
    else:
        falsche_links.append(lnk)

print(len(falsche_links))
print(len(richtige_links))


#iterating over all  pages and getting the relevant information into df
driver = webdriver.Chrome(PATH)

#creating the lists that eventually will form the dataframe

bundestags_id = []
wahlkreis = []
nachnamen = []
vornamen = []
facebook_links = []
instagram_links = []
twitter_links = []
website_links = []


# to verify the existence of elements

def check_link_exists(link_text):
    try:
        driver.find_element(By.CSS_SELECTOR, f'.bt-link-extern[title^="{link_text}"]')
    except NoSuchElementException:
        return False
    return True


webpage_exists = check_link_exists("Homepage")
facebook_exists = check_link_exists("Facebook")
twitter_exists = check_link_exists("Twitter")
instagram_exists = check_link_exists("Instagram")
wahlkreis_exists = check_link_exists("Wahlkreis")


#for test
#i = 0




for lnk in richtige_links:
    driver.get(lnk)
    
    try:
        #bundestags_id
        id_stripped = re.findall("\d+", lnk)
        bundestags_id.append(id_stripped[0])
        
        # names --- !!!! derzeit sind es nicht immer die korrekten namen. besser sie aus dem namenselement statt dem titel zu ziehen
        # NOCH UNKLAR: Welcher Name soll gewählt werden... in welchem Format. Hier scheinen Abweichungen auf der Website, bei manchen wird z.b. (Heilbronn) mitübergeben.
        get_title = driver.title
        nachnamen.append(re.findall("\w+", get_title)[-1])
        vornamen.append(re.findall("\w+", get_title)[-2])

        # wahlkreis
        if check_wahlkreis_exists() == True:
            search = driver.find_element(By.CSS_SELECTOR, '.bt-link-intern[title^="Wahlkreis"]').get_attribute('innerHTML')
            wahlkreis.append(search)
        else: #gibt mir das Bundesland
            search = driver.find_element(By.CSS_SELECTOR, '.bt-standard-content.col-sm-6.col-xs-12').get_attribute('innerHTML')
            land = re.findall('\>(.*)\<', search)
            wahlkreis.append('Landesliste:' + str(land))

        # links
        if check_webpage_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Homepage"]')
            website_links.append(element.get_attribute('href'))
        else:
            website_links.append('keine auf Bundestagsseite')

        if check_facebook_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Facebook"]')
            facebook_links.append(element.get_attribute('href'))
        else:
            facebook_links.append('keine auf Bundestagsseite')

        if check_twitter_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Twitter"]')
            twitter_links.append(element.get_attribute('href'))
        else:
            twitter_links.append('keine auf Bundestagsseite')
            
        if check_instagram_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Instagram"]')
            instagram_links.append(element.get_attribute('href'))
        else:
            instagram_links.append('keine auf Bundestagsseite')
       

    except:
        #driver.quit()
        print('duh!!!!!')
        print(error)
    
     # to test this loop with 5 iterations
   # i = i + 1
  #  if i == 5:
    #    break

df_politiker_selenium = pd.DataFrame()

df_politiker_selenium['bundestags_id'] = bundestags_id
df_politiker_selenium['vorname'] = vornamen
df_politiker_selenium['nachname'] = nachnamen
df_politiker_selenium['wahlkreis'] = wahlkreis
df_politiker_selenium['facebook_links'] = facebook_links
df_politiker_selenium['twitter_links'] = twitter_links
df_politiker_selenium['website_links'] = website_links
df_politiker_selenium['instagram_links'] = instagram_links

print(df_politiker_selenium)

end_time = time.time()
total_time = end_time - start_time

print("Time taken: ", total_time, "seconds")

18
758


C:\Users\grego\AppData\Local\Temp\ipykernel_11292\1701501393.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


duh!!!!!


NameError: name 'error' is not defined

In [5]:
# working with the links liste generated above. we need to filter out the wrong links first before iterating over it and fetching the relevant information

regex = 'biografien\/[a-zA-Z]\/'

falsche_links = []
richtige_links = []

for lnk in links:
    if bool(re.search(regex, lnk)):
        richtige_links.append(lnk)
    else:
        falsche_links.append(lnk)

print(len(falsche_links))
print(len(richtige_links))

#iterating over all  pages and getting the relevant information into df
driver = webdriver.Chrome(PATH)

#creating the lists that eventually will form the dataframe

bundestags_id = []
wahlkreis = []
nachnamen = []
vornamen = []
facebook_links = []
instagram_links = []
twitter_links = []
website_links = []


# to verify the existence of the element

def check_webpage_exists():
    try:
        driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Homepage"]')
    except NoSuchElementException:
        return False
    return True

def check_facebook_exists():
    try:
        driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Facebook"]')
    except NoSuchElementException:
        return False
    return True

def check_twitter_exists():
    try:
        driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Twitter"]')
    except NoSuchElementException:
        return False
    return True

def check_instagram_exists():
    try:
        driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Instagram"]')
    except NoSuchElementException:
        return False
    return True

def check_wahlkreis_exists():
    try:
        driver.find_element(By.CSS_SELECTOR, '.bt-link-intern[title^="Wahlkreis"]')
    except NoSuchElementException:
        return False
    return True

#for test
i = 0




for lnk in richtige_links:
    driver.get(lnk)
    
    try:
        #bundestags_id
        id_stripped = re.findall("\d+", lnk)
        bundestags_id.append(id_stripped[0])
        
        # names --- !!!! derzeit sind es nicht immer die korrekten namen. besser sie aus dem namenselement statt dem titel zu ziehen
        # NOCH UNKLAR: Welcher Name soll gewählt werden... in welchem Format. Hier scheinen Abweichungen auf der Website, bei manchen wird z.b. (Heilbronn) mitübergeben.
        get_title = driver.title
        nachnamen.append(re.findall("\w+", get_title)[-1])
        vornamen.append(re.findall("\w+", get_title)[-2])

        # wahlkreis
        if check_wahlkreis_exists() == True:
            search = driver.find_element(By.CSS_SELECTOR, '.bt-link-intern[title^="Wahlkreis"]').get_attribute('innerHTML')
            wahlkreis.append(search)
        else: #gibt mir das Bundesland
            search = driver.find_element(By.CSS_SELECTOR, '.bt-standard-content.col-sm-6.col-xs-12').get_attribute('innerHTML')
            land = re.findall('\>(.*)\<', search)
            wahlkreis.append('Landesliste:' + str(land))

        # links
        if check_webpage_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Homepage"]')
            website_links.append(element.get_attribute('href'))
        else:
            website_links.append('keine auf Bundestagsseite')

        if check_facebook_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Facebook"]')
            facebook_links.append(element.get_attribute('href'))
        else:
            facebook_links.append('keine auf Bundestagsseite')

        if check_twitter_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Twitter"]')
            twitter_links.append(element.get_attribute('href'))
        else:
            twitter_links.append('keine auf Bundestagsseite')
            
        if check_instagram_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Instagram"]')
            instagram_links.append(element.get_attribute('href'))
        else:
            instagram_links.append('keine auf Bundestagsseite')
       

    except:
        #driver.quit()
        print('duh!!!!!')
        print(error)
    
     # to test this loop with 5 iterations
    i = i + 1
    if i == 5:
        break

df_politiker_selenium = pd.DataFrame()

df_politiker_selenium['bundestags_id'] = bundestags_id
df_politiker_selenium['vorname'] = vornamen
df_politiker_selenium['nachname'] = nachnamen
df_politiker_selenium['wahlkreis'] = wahlkreis
df_politiker_selenium['facebook_links'] = facebook_links
df_politiker_selenium['twitter_links'] = twitter_links
df_politiker_selenium['website_links'] = website_links
df_politiker_selenium['instagram_links'] = instagram_links

print(df_politiker_selenium)

18
758


C:\Users\grego\AppData\Local\Temp\ipykernel_11292\3602561570.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


WebDriverException: Message: target frame detached
  (failed to check if window was closed: disconnected: Unable to receive message from renderer)
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x00BF37D3]
	(No symbol) [0x00B88B81]
	(No symbol) [0x00A8B230]
	(No symbol) [0x00A7D1BD]
	(No symbol) [0x00A7BC95]
	(No symbol) [0x00A7C63A]
	(No symbol) [0x00A85FE5]
	(No symbol) [0x00A9199E]
	(No symbol) [0x00A94DD6]
	(No symbol) [0x00A7C993]
	(No symbol) [0x00A91566]
	(No symbol) [0x00AF1758]
	(No symbol) [0x00ADB216]
	(No symbol) [0x00AB0D97]
	(No symbol) [0x00AB253D]
	GetHandleVerifier [0x00E6ABF2+2510930]
	GetHandleVerifier [0x00E98EC1+2700065]
	GetHandleVerifier [0x00E9C86C+2714828]
	GetHandleVerifier [0x00CA3480+645344]
	(No symbol) [0x00B90FD2]
	(No symbol) [0x00B96C68]
	(No symbol) [0x00B96D4B]
	(No symbol) [0x00BA0D6B]
	BaseThreadInitThunk [0x770000F9+25]
	RtlGetAppContainerNamedObjectPath [0x77487BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77487B8E+2

In [21]:
class LinkFiltering:
    def __init__(self, richtige_links):
        self.links = links

    def filter_links(self):
        regex = 'biografien\/[a-zA-Z]\/'
        falsche_links = []
        richtige_links = []
        for lnk in self.links:
            if bool(re.search(regex, lnk)):
                richtige_links.append(lnk)
            else:
                falsche_links.append(lnk)
        return richtige_links
    

In [26]:
class PoliticianData:
    
    def __init__(self, richtige_links, path):
        self.links = links
        self.path = path
        self.driver = webdriver.Chrome(self.path)
        self.bundestags_id = []
        self.wahlkreis = []
        self.nachnamen = []
        self.vornamen = []
        self.facebook_links = []
        self.instagram_links = []
        self.twitter_links = []
        self.website_links = []
        
    def __del__(self):
        self.driver.quit()
        
    def filter_links(self, regex='biografien\/[a-zA-Z]\/'):
        falsche_links = []
        richtige_links = []

        for lnk in self.links:
            if bool(re.search(regex, lnk)):
                richtige_links.append(lnk)
            else:
                falsche_links.append(lnk)

        self.links = richtige_links
        
    def check_webpage_exists(self):
        try:
            self.driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Homepage"]')
        except NoSuchElementException:
            return False
        return True

    def check_facebook_exists(self):
        try:
            self.driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Facebook"]')
        except NoSuchElementException:
            return False
        return True

    def check_twitter_exists(self):
        try:
            self.driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Twitter"]')
        except NoSuchElementException:
            return False
        return True

    def check_instagram_exists(self):
        try:
            self.driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Instagram"]')
        except NoSuchElementException:
            return False
        return True

    def check_wahlkreis_exists(self):
        try:
            self.driver.find_element(By.CSS_SELECTOR, '.bt-link-intern[title^="Wahlkreis"]')
        except NoSuchElementException:
            return False
        return True
    
    def get_politician_data(self):
        self.driver.get(self.profile_link)

    def check_element_exists(selector):
        try:
            element = self.driver.find_element(By.CSS_SELECTOR, selector)
        except NoSuchElementException:
            return False
        return element

    try:
        #bundestags_id
        id_stripped = re.findall("\d+", self.profile_link)
        self.bundestags_id = id_stripped[0]

        # names
        title = check_element_exists('.bt-standard-header__headline')
        if title:
            names = re.findall("\w+", title.get_attribute('innerHTML'))
            self.nachname = names[-1]
            self.vorname = names[-2]

        # wahlkreis
        wahlkreis = check_element_exists('.bt-link-intern[title^="Wahlkreis"]')
        if wahlkreis:
            self.wahlkreis = wahlkreis.get_attribute('innerHTML')
        else:
            land = check_element_exists('.bt-standard-content.col-sm-6.col-xs-12')
            if land:
                self.wahlkreis = 'Landesliste:' + re.findall('\>(.*)\<', land.get_attribute('innerHTML'))

        # links
        self.website_links = check_element_exists('.bt-link-extern[title^="Homepage"]').get_attribute('href') or 'keine auf Bundestagsseite'
        self.facebook_links = check_element_exists('.bt-link-extern[title^="Facebook"]').get_attribute('href') or 'keine auf Bundestagsseite'
        self.twitter_links = check_element_exists('.bt-link-extern[title^="Twitter"]').get_attribute('href') or 'keine auf Bundestagsseite'
        self.instagram_links = check_element_exists('.bt-link-extern[title^="Instagram"]').get_attribute('href') or 'keine auf Bundestagsseite'

    except Exception as e:
        print(f'Error occurred while processing profile link: {self.profile_link}.\nError message: {str(e)}')


NameError: name 'self' is not defined